# <span style="color:#8B4F1D">2D Decaying Isotropic Turbulence </span>
##
## - Governing equation: Vorticity streamfunction formulation
###
### * Navier-Stokes equations in convective form
<center><span style="font-size:18pt"> momentum equation: $\frac{Du_{i}}{Dt} = \frac{\partial u_i}{\partial t} + u_j\frac{\partial u_i}{\partial x_j} = -\frac{1}{\rho}\frac{\partial p}{\partial x_i} + \nu\frac{\partial^2 u_i}{\partial x_j \partial x_j}$ &nbsp; & &nbsp; continuity equation: $\frac{\partial \rho}{\partial t} + \frac{\partial \left(\rho u_i\right)}{\partial x_i} = \frac{\partial u_i}{\partial x_i} = 0$ </span></center>  <br>

### * Vorticity-streamfunction formulation
<center><span style="font-size:15pt"> Taking curl of the N-S eqn.s yields,</span><br>
<span style="font-size:18pt"> vorticity transport equation: $\frac{D\omega}{Dt} = \frac{\partial \omega}{\partial t} + u_j\frac{\partial \omega}{\partial x_j} = \nu\frac{\partial^2 \omega}{\partial x_j \partial x_j}$ &nbsp; & &nbsp; streamfunction formulation: $\frac{\partial^2 \psi}{\partial x_j \partial x_j} = -\omega$ </span></center>  <br>


## - Numerical methods
###
### 1. Spatial discretization
- Domain: $[0,2\pi)^2$,&nbsp; Number of grids: $128 \times 128$ uniform grid
- Bi-periodic boundary condition
- Pesudo-spectral method with 3/2 zero-padding for de-aliasing
###
### 2. Time advancement
- Time step size: $dt=0.0025$,&nbsp; Number of time steps: 12,000 (until T = 30)
- Random (normal) initial condition $\to$ random phase fixed amplitude
- Crank-Nicolson method for viscous term and 2nd order Adams-Bashforth method for convective term

In [1]:
### Removing interpolation ###
def zero_padding(X):
    X[:,nx//2:] = 0
    X[ny//2:my-(ny//2-1),:] = 0

def removing_interpolation(X):
    Y = np.concatenate((X[:ny//2,:nx//2+1],X[my-ny//2:,:nx//2+1]), axis = 0)
    return Y

def get_nonlinear_wave(wk):
    psik[0:,1:] = wk[0:,1:]/k_mag[0:,1:]
    psik[1:,0:1] = wk[1:,0:1]/k_mag[1:,0:1]

    psi_xk = 1J*k1*psik; psi_yk = 1J*k2*psik
    psi_x = np.fft.irfft2(psi_xk)*(mx*my); psi_y = np.fft.irfft2(psi_yk)*(mx*my)
    w_x = np.fft.irfft2(1J*k1*wk)*(mx*my); w_y = np.fft.irfft2(1J*k2*wk)*(mx*my)
    
    nonlinear_wave = (np.fft.rfft2(psi_y*w_x-psi_x*w_y))/(mx*my)
    zero_padding(nonlinear_wave)
    return nonlinear_wave, psi_xk, psi_yk

def get_velocity(psi_xk, psi_yk):
    psi_xk_real = removing_interpolation(psi_xk); psi_yk_real = removing_interpolation(psi_yk)
    psi_x_real = np.fft.irfft2(psi_xk_real)*(nx*ny); psi_y_real = np.fft.irfft2(psi_yk_real)*(nx*ny)
    return psi_y_real, -psi_x_real

def dissipation(psi_xk, psi_yk):
    psi_xxk = 1J*k1*psi_xk; psi_yyk = 1J*k2*psi_yk
    psi_xyk = 1J*k2*psi_xk; psi_yxk = 1J*k1*psi_yk
    psi_xxk_real = removing_interpolation(psi_xxk); psi_yyk_real = removing_interpolation(psi_yyk);
    psi_xyk_real = removing_interpolation(psi_xyk); psi_yxk_real = removing_interpolation(psi_yxk);
    
    psi_xx_real = np.fft.irfft2(psi_xxk_real)*(nx*ny); psi_yy_real = np.fft.irfft2(psi_yyk_real)*(nx*ny)
    psi_xy_real = np.fft.irfft2(psi_xyk_real)*(nx*ny); psi_yx_real = np.fft.irfft2(psi_yxk_real)*(nx*ny)
    d = nu*np.mean(psi_xx_real**2 + psi_yy_real**2 + psi_xy_real**2 + psi_yx_real**2)
    return d

def kolmogorov_scale(d):
    eta = (nu**3/d)**(1/4)
    return eta

In [1]:
import numpy as np
import pandas as pd
import h5py
import sys

pi = np.pi; nu = 10**-3
nx = 128; ny = 128; mx = 3*nx//2; my = 3*ny//2; a = 0; b = 2*pi; L = b-a; T = 3*10**1; dt = 2.5*10**-3; n = int(T/dt); K = 20
# nx = 96; ny = 96; mx = 3*nx//2; my = 3*ny//2; a = 0; b = 2*pi; L = b-a; T = 5*10**1; dt = 5*10**-3; n = int(T/dt); K = 10

Directory = 'C:/Users/KJY/Desktop/Jupyter/Spectral method/2D isotropic/resolution 128/Test data'

x = np.linspace(a, b, nx+1, dtype = np.float64)
y = np.linspace(a, b, ny+1, dtype = np.float64)
t = np.linspace(0, T, n+1, dtype = np.float64)
k1 = np.zeros([1,mx//2+1], dtype = np.float64)
k2 = np.zeros([my,1], dtype = np.float64)
psik = np.zeros([my,mx//2+1], dtype = np.complex128)
w = np.zeros([n//K+1,ny,nx,1], dtype = np.float64)
u = np.zeros([n//K+1,ny,nx,1], dtype = np.float64)
v = np.zeros([n//K+1,ny,nx,1], dtype = np.float64)
dissipation_rate = np.zeros([n//K+1], dtype = np.float64)
kolmogorov_length = np.zeros([n//K+1], dtype = np.float64)

In [31]:
print(n//K+1)

601


In [ ]:
import numpy as np

for px in range(mx//2+1):
    k1[0,px] = (2*pi/L)*px # 마지막 wavenubmer component는 필요없는 듯? 아마도 zero padding 됨.
for py in range(my):
    if py < my/2:
        k2[py,0] = (2*pi/L)*py
    else:
        k2[py,0] = (2*pi/L)*py-my
k_mag = k1**2+k2**2

(np.abs(k_mag-10.0)+1

In [75]:
for px in range(mx//2+1):
    k1[0,px] = (2*pi/L)*px # 마지막 wavenubmer component는 필요없는 듯? 아마도 zero padding 됨.
for py in range(my):
    if py < my/2:
        k2[py,0] = (2*pi/L)*py
    else:
        k2[py,0] = (2*pi/L)*py-my
k_mag = k1**2+k2**2

for num in range(50):
    num += 1
    
    w0 = np.random.normal(loc = 0.0, scale = 1.0, size = (my,mx))
    wk = np.fft.rfft2(w0)/(mx*my)
    wk = wk/(np.abs(wk)+10**-5) # 모든 wave number에 대한 amplitude를 1로 맞춰줌. 10**-5 더해주는 이유는 0인 경우 에러가 나는 거 방지
    wk = ((np.abs(k_mag-10.0)+1)**-2.0)*wk
    zero_padding(wk) ; wk[0,0] = 0
    
    Jkp, psi_xk, psi_yk = get_nonlinear_wave(wk)
    Jk = Jkp
    
    wk_real = removing_interpolation(wk)
    wn = np.fft.irfft2(wk_real)*(nx*ny)
    un, vn = get_velocity(psi_xk, psi_yk)
    w[0,:,:,0] = wn[:,:]; u[0,:,:,0] = un[:,:]; v[0,:,:,0] = vn[:,:]
    dissipation_rate[0] = dissipation(psi_xk, psi_yk); kolmogorov_length[0] = kolmogorov_scale(dissipation_rate[0])
    
    for q in range(1,n+1):
        wk = ((1-0.5*nu*k_mag*dt)*wk - dt*(1.5*Jk-0.5*Jkp))/(1+0.5*nu*k_mag*dt)
        zero_padding(wk) ; wk[0,0] = 0

        Jkn, psi_xk, psi_yk = get_nonlinear_wave(wk)
        if q == 1:
            Jk = Jkn
        else:
            Jkp = Jk ; Jk=Jkn

        if q % K == 0:
            wk_real = removing_interpolation(wk)
            wn = np.fft.irfft2(wk_real)*(nx*ny)
            un, vn = get_velocity(psi_xk, psi_yk)
            w[q//K,:,:,0] = wn[:,:]; u[q//K,:,:,0] = un[:,:]; v[q//K,:,:,0] = vn[:,:]
            dissipation_rate[q//K] = dissipation(psi_xk, psi_yk); kolmogorov_length[q//K] = kolmogorov_scale(dissipation_rate[q//K])
#             print(dissipation_rate[q//K], kolmogorov_length[q//K])
    
    CreateData = Directory + '/Test%d 2D HIT n=%d T=%d dt=%.4lf K=%d data.h5' %(num,nx,T,dt,K)
    fc = h5py.File(CreateData, 'w')
    fc.create_dataset('w', data = w) # 여기 w는 write한다는 말이 아니고 변수명 오메가 대신 쓴 w
    fc.close()
    
    print('%d simulation is done.\n' %num)

1 simulation is done.

2 simulation is done.

3 simulation is done.

4 simulation is done.

5 simulation is done.

6 simulation is done.

7 simulation is done.

8 simulation is done.

9 simulation is done.

10 simulation is done.

11 simulation is done.

12 simulation is done.

13 simulation is done.

14 simulation is done.

15 simulation is done.

16 simulation is done.

17 simulation is done.

18 simulation is done.

19 simulation is done.

20 simulation is done.

21 simulation is done.

22 simulation is done.

23 simulation is done.

24 simulation is done.

25 simulation is done.

26 simulation is done.

27 simulation is done.

28 simulation is done.

29 simulation is done.

30 simulation is done.

31 simulation is done.

32 simulation is done.

33 simulation is done.

34 simulation is done.

35 simulation is done.

36 simulation is done.

37 simulation is done.

38 simulation is done.

39 simulation is done.

40 simulation is done.

41 simulation is done.

42 simulation is done.

4

In [82]:
Data_dir = 'C:/Users/KJY/Desktop/Jupyter/Spectral method/2D isotropic/resolution 128/Test data'
ReadData = Data_dir + '/Test%d 2D HIT n=%d T=%d dt=%.4lf K=%d data.h5' %(num,nx,T,dt,K)
fr = h5py.File(ReadData, 'r')
w = np.reshape(fr['w'][:], [n//K+1,ny,nx,1])
fr.close()

Filename = Directory + '/Test%d 2D HIT n=%d T=%d dt=%.4lf K=%d.plt' %(num,nx,T,dt,K)
fw = open(Filename, 'w')
fw.write('VARIABLES="x/pi","y/pi","w"\n')
for q in range(n//K+1):
    fw.write('ZONE T="T=%.4lf" I=%d J=%d\n' %(t[q*K],nx,ny))
    for j in range(ny):
        for i in range(nx):
            fw.write('%lf %lf %lf\n' %(x[i]/pi,y[j]/pi,w[q,j,i,0]))
fw.close()

49


In [12]:
import numpy as np
import pandas as pd
import h5py
import sys
import cv2

pi = np.pi; nu = 10**-3
nx = 128; ny = 128; mx = 3*nx//2; my = 3*ny//2; a = 0; b = 2*pi; L = b-a; T = 3*10**1; dt = 2.5*10**-3; n = int(T/dt); K = 20
x = np.linspace(a, b, nx+1, dtype = np.float64)
y = np.linspace(a, b, ny+1, dtype = np.float64)
t = np.linspace(0, T, n+1, dtype = np.float64)

Data_dir = 'D:/대학원/연구 주제/2. PANN/연구 진행/Data/resolution 128/Training data'
w = np.zeros([5,n//K+1,ny,nx,1], dtype = np.float64)
w_up = np.zeros([5,n//K+1,2*ny,2*nx], dtype = np.float64)
for num in range(5):
    ReadData = Data_dir + '/Training%d 2D HIT n=%d T=%d dt=%.4lf K=%d data.h5' %(num+1,nx,T,dt,K)
    fr = h5py.File(ReadData, 'r')
    w[num,:] = np.reshape(fr['w'][:], [n//K+1,ny,nx,1])
    fr.close()
w_resize = np.reshape(w,[5,n//K+1,ny,nx])

for i in range(5):
    for j in range(n//K+1):
        w_up[i,j,:] = cv2.resize(w_resize[i,j,:], (2*ny,2*nx), interpolation = cv2.INTER_LINEAR) 
w_up = np.reshape(w_up,[5,n//K+1,2*ny,2*nx,1])

Direc = 'C:/Users/KJY/Dropbox/Courses/Graduate school/Coursework/2021 2학기/Generative Adversarial Networks/Homeworks'
Directory = Direc + '/test experiments/Experiment/FreezeD-master'
for num in range(5):
    CreateData = Directory + '/Training%d 2D HIT n=%d T=%d dt=%.4lf K=%d data.h5' %(num+1,2*nx,T,dt,K)
    fc = h5py.File(CreateData, 'w')
    fc.create_dataset('w', data = w_up) # 여기 w는 write한다는 말이 아니고 변수명 오메가 대신 쓴 w
    fc.close()

In [8]:
w_resize = np.reshape(w,[5,n//K+1,ny,nx])
for i in range(5):
    for j in range(n//K+1):
        w_resize[i,j,:] = cv2.resize(w_resize[i,j,:], (2*ny,2*nx), interpolation = cv2.INTER_LINEAR) 
print(np.shape(w_resize))

ValueError: could not broadcast input array from shape (256,256) into shape (128,128)

In [77]:
Filename4 = Directory + '/Test%d 2D HIT n=%d T=%d dt=%.4lf K=%d dissipation rate.plt' %(num,nx,T,dt,K)
fdissipation = open(Filename4, 'w')
fdissipation.write('VARIABLES="t","epsilon"\n')
fdissipation.write('ZONE T="dissipation rate" I=%d\n' %(n//K+1))
for q in range(n//K+1):
    fdissipation.write('%lf %lf\n' %(t[q*K],dissipation_rate[q]))
fdissipation.close()

Filename5 = Directory + '/Test%d 2D HIT n=%d T=%d dt=%.4lf K=%d kolmogorov length scale.plt' %(num,nx,T,dt,K)
fkolmogorov = open(Filename5, 'w')
fkolmogorov.write('VARIABLES="t","eta"\n')
fkolmogorov.write('ZONE T="kolmogorov length scale" I=%d\n' %(n//K+1))
for q in range(n//K+1):
    fkolmogorov.write('%lf %lf\n' %(t[q*K],kolmogorov_length[q]))
fkolmogorov.close()

In [78]:
w_rms = np.sqrt(np.mean(w**2, axis = (1,2)))
u_rms = np.sqrt(np.mean(u**2, axis = (1,2))); v_rms = np.sqrt(np.mean(v**2, axis = (1,2))); U_rms=(u_rms+v_rms)/2

Filename1 = Directory +'/Test%d 2D HIT n=%d T=%d dt=%.4lf K=%d w and U rms.plt' %(num,nx,T,dt,K)
frms = open(Filename1, 'w')
frms.write('VARIABLES="t","w_rms","U_rms"\n')
frms.write('ZONE T="Root mean squares" I=%d\n' %(n//K+1))
for q in range(n//K+1):
    frms.write('%lf %lf %lf\n' %(t[q*K],w_rms[q],U_rms[q]))
frms.close()

In [79]:
mean_ins = np.mean(w, axis = (1,2))
std_ins = np.std(w,axis = (1,2))
w_fluc = w-np.mean(w, axis = (1,2), keepdims = True)
skew_ins = np.mean(w_fluc**3, axis = (1,2))/std_ins**3
kurto_ins = np.mean(w_fluc**4, axis = (1,2))/std_ins**4

Filename2 = Directory + '/Test%d 2D HIT n=%d T=%d dt=%.4lf K=%d statistics.plt' %(num,nx,T,dt,K)
fs = open(Filename2, 'w')
fs.write('VARIABLES="t","mean_ins","std_ins","skew_ins","kurto_ins"\n')
fs.write('ZONE T="Statistics" I=%d\n' %(n//K+1))
for q in range(n//K+1):
    fs.write('%lf %lf %lf %lf %lf\n' %(t[q*K],mean_ins[q],std_ins[q],skew_ins[q],kurto_ins[q]))
fs.close()

In [27]:
Training_dir = 'C:/Users/KJY/Desktop/Jupyter/Spectral method/2D isotropic/Training data'
rho = np.zeros([500-150+1], dtype=np.float64)
tau_bar_avg = 0
for num in range(200):
    num += 1
    
    ReadData = Training_dir + '/Training%d 2D HIT n=%d T=%d dt=%.4lf K=%d data.h5' %(num,nx,T,dt,K)
    fr = h5py.File(ReadData, 'r')
    read = np.reshape(fr['w'][:], [n//K+1,my,mx,1])
    fr.close()
    
    w_fluc = read - np.mean(read, axis=(1,2), keepdims=True)
    w_fluc_normalized = w_fluc/np.std(read, axis=(1,2), keepdims=True)

#     Filename3 = Directory +'/Training%d 2D HIT n=%d T=%d dt=%.4lf K=%d auto-correlation.plt' %(num,nx,T,dt,K)
#     fa = open(Filename3, 'w')
#     fa.write('VARIABLES="s","rho"\n')
#     fa.write('ZONE T="Auto-correlation" I=%d\n' %(500-150+1))
    for q in range(500-150+1):
        rho[q] = np.mean(w_fluc_normalized[101,:,:,:]*w_fluc_normalized[101+q,:,:,:])
#         fa.write('%lf %lf \n' %((dt*K)*q,rho[q]))
#     fa.close()

    tau_bar = 0
    for q in range(1,500-150+1):
        tau_bar += (rho[q-1]+rho[q])/2*(dt*K)
    tau_bar_avg += tau_bar
    print(tau_bar)

    if num == 200:
        print(rho)
tau_bar_avg = tau_bar_avg/200
print('\n',tau_bar_avg)
# print(np.shape(rho))
# print(np.shape(data - np.mean(data, axis = 1)))
# y_fluc_normalized = (data - np.mean(data, axis = 1))/np.std(data[])
# print(np.mean((data[100]-np.mean(data[100]))*(data[101]-np.mean(data[101]))))
# print(np.shape(np.mean(y_fluc_normalized[1000:1500,:,:], axis = 0)))
# print(np.shape(np.mean(data[100:150,:,:], axis = 0)))

1.6576563658552004
1.3827508108089206
1.975078483562751
3.051470362717899
2.3597729576428033
1.0945066739028155
1.8832383833303377
1.0722517450681737
1.5611212549119553
0.7425593062422596
0.8023485547790288
1.7540482905260628
2.8424047228750435
1.299867950811571
1.3630006722070387
1.5557202102549046
1.423559113059278
2.749570338747951
2.0528204752248547
2.041649351716103
1.7036011111557798
1.244459158202407
3.35223285101372
2.911506522546474
1.9117040344734337
1.9927437689650707
1.9506409826118976
1.7057842015995641
2.7250357256685853
1.6770968925471954
1.60212836469717
1.5434705346371462
1.8657929925642667
2.7522026278318905
1.680021899912927
1.511456312751433
2.1405865059187335
2.130303268059634
1.2346504707939305
1.2322687542112742
1.3674189003059074
2.3316059778935028
0.6825412225172011
2.6016368091955324
1.8861081361336747
1.0250154039759702
1.8024430361064299
0.8758547901760426
1.2815106269490273
2.3161032995135336
2.623811382763449
0.8240458541572848
2.5628888098778235
2.7000930

In [28]:
print(tau_bar_avg)

1.7869437782420872


In [ ]:
fr=h5py.File(CreateData, 'r')
read = fr['w'][:]
fr.close()
print(np.shape(read))
np.allclose(w,read)

In [ ]:
# 초기 #
def zero_padding(X):
    X[:,nx//2:] = 0
    X[ny//2:my-(ny//2-1),:] = 0

def get_nonlinear_wave(wk):
    psik[0:,1:] = wk[0:,1:]/k_mag[0:,1:]
    psik[1:,0:1] = wk[1:,0:1]/k_mag[1:,0:1]

    psi_xk = 1J*k1*psik
    psi_yk = 1J*k2*psik
    psi_x = np.fft.irfft2(psi_xk)*(mx*my)
    psi_y = np.fft.irfft2(psi_yk)*(mx*my)
    w_x = np.fft.irfft2(1J*k1*wk)*(mx*my)
    w_y = np.fft.irfft2(1J*k2*wk)*(mx*my)
    nonlinear_wave = (np.fft.rfft2(psi_y*w_x-psi_x*w_y))/(mx*my)
    
    zero_padding(nonlinear_wave)
    return nonlinear_wave, psi_y, -psi_x

In [ ]:
def HIT_solver(u, n_time_step=rl_nt, dt=rl_dt):
    u = np.float64(u)
    u_gpu = cp.asarray(u)
    up_gpu = cp.fft.rfft2(u_gpu, axes=(0,1))/(nxp*nyp)

    for t in range(n_time_step):
        zero_padding(up_gpu)
        vorp_gpu = 1j*alpha_gpu*up_gpu[:,:,1:2] - 1j*beta_gpu*up_gpu[:,:,0:1] 
        u_gpu = cp.fft.irfft2(up_gpu, axes=(0,1))*(nxp*nyp)
        vor_gpu = cp.fft.irfft2(vorp_gpu, axes=(0,1))*(nxp*nyp)
        
        H_gpu = cp.concatenate([u_gpu[:,:,1:2]*vor_gpu,-u_gpu[:,:,0:1]*vor_gpu],axis=2)
        Hp_gpu = cp.fft.rfft2(H_gpu, axes=(0,1))/(nxp*nyp)
        
        kHp_gpu = alpha_gpu*Hp_gpu[:,:,0:1]+beta_gpu*Hp_gpu[:,:,1:2]
        kkHp_gpu = cp.concatenate([alpha_gpu*kHp_gpu,beta_gpu*kHp_gpu], axis=2)

        NLp1_gpu[:,:,:] = NLp2_gpu[:,:,:]
        NLp2_gpu[:,:,:] = -1.0/k_square_gpu*kkHp_gpu+Hp_gpu[:,:,0:2]

        if t==0 : NLp1_gpu[:,:,:] = NLp2_gpu[:,:,:]

        up_gpu[:,1:,0:2] = 1.0/(1.0+nu*k_square_gpu[:,1:]*dt/2.0)*\
                             ((1.0-nu*k_square_gpu[:,1:]*dt/2.0)*up_gpu[:,1:]
                              +dt*(1.5*NLp2_gpu[:,1:]-0.5*NLp1_gpu[:,1:]))
        up_gpu[1:,0:1,0:2] = 1.0/(1.0+nu*k_square_gpu[1:,0:1]*dt/2.0)*\
                             ((1.0-nu*k_square_gpu[1:,0:1]*dt/2.0)*up_gpu[1:,0:1]
                              +dt*(1.5*NLp2_gpu[1:,0:1]-0.5*NLp1_gpu[1:,0:1]))
        up_gpu[0:1,0:1,:] = 0.0

    zero_padding(up_gpu)
    u_gpu = cp.fft.irfft2(up_gpu, axes=(0,1))*(nxp*nyp)
    u = cp.asnumpy(u_gpu)
    return u